In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import iqr
import pickle
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Activation

# from google.colab import drive
# drive.mount('/content/drive')
path_data = 'data/'
path_model = 'model/'

X_train = np.load(path_data + 'X_train.npy')
y_train = np.load(path_data + 'y_train.npy')
X_test = np.load(path_data + 'X_test.npy')
y_test = np.load(path_data + 'y_test.npy')
print(X_train.shape,y_train.shape, np.min(X_train),np.max(X_train),np.min(y_train),np.max(y_train))

with open(path_data + "sc.pickle","rb") as fr:
    sc1 = pickle.load(fr)
data_hr1 = np.load(path_data + 'data_hr1.npy')

n_steps_in=24*3
n_steps_out=24
n_step=n_steps_in
n_features = X_train.shape[-1]

## Network

In [ ]:
model = Sequential()
#model.add(LSTM(256, activation='relu',dropout=0.5, input_shape=(n_steps_in, n_features)))#9
model.add(LSTM(256, activation='relu',input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(256, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(256, activation='relu')))
#model.add(TimeDistributed(Dense(50, activation='relu')))
model.add(TimeDistributed(Dense(1)))

In [ ]:
# Train
cp_callback = ModelCheckpoint(filepath = path_model+'model.h5', save_weights_only=True, verbose=1)
model.compile(loss='mse', optimizer='rmsprop')
model.fit(X_train,y_train[:,:,0:1], batch_size=64, epochs=500, validation_data=(X_test,y_test[:,:,:1]),callbacks=[cp_callback])
model.load_weights(path_model+'model.h5')

## Result

In [ ]:
pred1= model.predict(X_test)
pred1=pred1.reshape(-1,1)
pred1 = sc1.inverse_transform(pred1)
pred1=pred1[:,0]
print(pred1.shape)

fig = plt.figure(figsize=(20,3))                                                               
ax = fig.add_subplot(1,1,1)                                                      

# This is where I manually set ticks. Can I use Datetime data instead???                                     
major_ticks = np.arange(0, len(pred1)+1, 24)                                              
minor_ticks = np.arange(0, len(pred1)+1, 1)                                               

gt = data_hr1[-len(pred1):,0]
plt.plot(gt,marker='o',ls='--',c='orange', label = 'Real')
ax.plot(pred1, 'bo-', label = 'Pred') 
ax.legend()

ax.set_xticks(major_ticks)                                                       
ax.set_xticks(minor_ticks, minor=True)                                           
#ax.set_yticks(major_ticks)                                                       
#ax.set_yticks(minor_ticks, minor=True)                                           
ax.grid(which='both')                                                                                           
ax.grid(which='minor', alpha=0.3)                                                
ax.grid(which='major', alpha=1.0,linewidth=1.8,color='r',axis='x')
# plt.savefig('tmp.png')
plt.show()

# error
r = np.sqrt(mean_squared_error(pred1, gt))
print(r)
print(r/np.max(gt))